## Install Libraries

In [ ]:
!pip install -U bitsandbytes
!pip install -U peft
!pip install -U trl
!pip install -U accelerate
!pip install -U datasets
!pip install -U transformers
!pip install -U wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 19.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Succes

In [ ]:
import torch
import pandas as pd
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

## Data Load

In [ ]:
design_data = pd.read_csv('/content/drive/MyDrive/Gemma/Data/Design/Design_data.csv')
design_data.head()

,성별,나이대,신입,질문,답변
0,여성,30대,경력,디자이너로서 앞으로의 목표에 관해서 설명해 주세요,저는 디자이너로서 어 많은 경험들을 쌓아서 그것들을 토대로 제 개인 브랜드를 런칭하...
1,여성,30대,경력,협업을 할 때 사교성이 좋은 편인지 궁금합니다 그리고 또 사교성을 키우기 위해 어떤...,저의 큰 단점이라고 할 수 있는데요. 저는 굉장히 성격이 내성적이고 처음 본 사람하...
2,여성,30대,경력,지원자님이 태어나서 지금까지 한 일들 가운데 가장 후회했던 일이 무엇인가요 한 가지...,어렸을 때 굉장히 친하게 지낸 친구가 있습니다. 어 제일 친하다고 생각했었고 모든 ...
3,여성,30대,경력,대학 생활을 보내면서 가장 힘들었던 경험은 무엇인가요 그것을 어떻게 극복하였는지 예...,저는 등록금을 마련하는 게 가장 힘들었습니다. 집에서 저의 대학 등록금을 마련해 줄...
4,여성,30대,경력,가장 자신있는 작업은 무엇일까요,저는 손으로 만드는 것은 다 자신 있습니다. 깊게 배우지 않아도 어 뭔가를 만드는 ...


In [ ]:
design_data['답변'][0]

'저는 디자이너로서 어 많은 경험들을 쌓아서 그것들을 토대로 제 개인 브랜드를 런칭하는 게 목표입니다. 어 회사에서 생활을 하다 보면 저의 개인적인 개성이 많이 무시당한다고 느낀 적이 많았습니다. 그래서 배웠던 것들을 토대로 제 개성을 살릴 수 있는 디자인 브랜드를 내고 싶습니다. 그리고 그 브랜드를 엄청 욕심 있게 크게 키우는 것이 아니라 제 혼자서 감당할 수 있는 선까지 키울 수 있는 그런 미래를 생각하고 있습니다.'

In [ ]:
design_data.shape

(6525, 5)

In [ ]:
design_vdata = pd.read_csv('/content/drive/MyDrive/Gemma/Data/Design/Design_vdata.csv')
design_vdata.head()

,성별,나이대,신입,질문,답변
0,여성,50대,경력,매일 공부하고 있는 분야가 있나요,어 매일 공부하고 있는 분야가 있나요 라고 하셨는데요. 저는 어 공부하는 것이 있습...
1,여성,50대,경력,대학 생활 가장 힘들었던 경험은 무엇인가요,어 대학 생활 중에서 가장 힘들었던 경험은 무엇인가요 라고 하셨는데요. 저는 방송통...
2,여성,50대,경력,지금 제일 가고 싶은 회사는,지금 제일 가고 싶은 회사는 이라고 질문을 주셨는데요. 물론 제가 지금 일하고 있는...
3,여성,50대,경력,아 기존 회사를 그만둔 것 퇴사한 것에 대해서 현재 후회가 없으신지 궁금하네요 어 ...,퇴사한 것을 후회하지 않으시나요 라고 질문을 주셨는데 어 간혹 가다가 그만두기 전에...
4,여성,50대,경력,어 내가 직무를 수행함에 있어서 어떤 뭐 모르는 과제도 나올 수 있겠죠 그런 문제에...,직무 수행 시 모르는 과제에 대해서 해결하는 방법은 무엇입니까 라고 질문을 주셨는데...


In [ ]:
design_vdata.shape

(813, 5)

In [ ]:
print('Design 데이터의 훈련 데이터의 퍼센트: ', (design_data.shape[0] / (design_data.shape[0] + design_vdata.shape[0])) * 100 )

Design 데이터의 훈련 데이터의 퍼센트:  88.92068683565005


## Huggingface Login

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

## Modeling

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GemmaTokenizer

model_id = "google/gemma-2-2b-it"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map='auto',
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [ ]:
# CUDA 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
from peft import LoraConfig

lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

In [ ]:
design_dataset = Dataset.from_pandas(design_data)
design_dataset

Dataset({
    features: ['성별', '나이대', '신입', '질문', '답변'],
    num_rows: 6525
})

In [ ]:
import wandb

wandb.login()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
run = wandb.init(
    project='Fine tuning gemma2 Design',
    job_type="training",
    anonymous="allow"
)

train/epoch,▁▅█
train/global_step,▁▅█
train/grad_norm,▁█▇
train/learning_rate,▁▄█
train/loss,▁█▃
train/epoch,0.00368
train/global_step,3
train/grad_norm,0.78366
train/learning_rate,1e-05
train/loss,2.7443


In [ ]:
new_model = "/content/drive/MyDrive/Gemma/results/Fine tuning gemma Design"

In [ ]:
import transformers
from trl import SFTTrainer,DataCollatorForCompletionOnlyLM

def formatting_func(example): # example을 받아서 text로 변환
    output_texts =[]
    for i in range(len(example['질문'])):
        text = f"### {example['나이대'][i]}의 {example['신입'][i]}의 {example['성별'][i]}에게 질문: {example['질문'][i]} \n ### 답변: {example['답변'][i]}"
        output_texts.append(text)
    return output_texts

response_template = " ### 답변:"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

training_arguments=transformers.TrainingArguments(
        output_dir=new_model,
        per_device_train_batch_size=1,
        optim="paged_adamw_32bit",
        gradient_accumulation_steps= 8,
        num_train_epochs=2,
        logging_steps=1,
        warmup_steps=10,
        logging_strategy="steps",
        learning_rate=2e-5,
        fp16=False,
        bf16=False,
        group_by_length=True,
        report_to="wandb"
    )

trainer = SFTTrainer(  # Supervised Fine-tuning Trainer
    model=model,
    train_dataset=design_dataset,
    args=training_arguments,
    peft_config=lora_config,
    formatting_func=formatting_func,
    data_collator=collator,
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/6525 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:396: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


Step,Training Loss
1,2.729300
2,2.787000
3,2.744400
4,2.648900
5,2.611300
6,2.507900
7,2.731300
8,2.681700
9,2.740300
10,2.472400


Step,Training Loss
1,2.729300
2,2.787000
3,2.744400
4,2.648900
5,2.611300
6,2.507900
7,2.731300
8,2.681700
9,2.740300
10,2.472400


TrainOutput(global_step=1630, training_loss=2.150778710915267, metrics={'train_runtime': 19512.2371, 'train_samples_per_second': 0.669, 'train_steps_per_second': 0.084, 'total_flos': 5.320926935011584e+16, 'train_loss': 2.150778710915267, 'epoch': 1.9984674329501915})

In [ ]:
wandb.finish()

train/epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇████
train/global_step,▁▁▁▁▁▁▂▂▂▃▃▃▃▃▃▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇███
train/grad_norm,▂▂▂▂▁▂▃▄▄▄▃▃▄▄▄▄▄▅▄▅▅▄▆▄▄▆▅▅▆▇▅▆▇▅▅▇▇█▅▅
train/learning_rate,████▇▇▇▇▇▆▆▆▆▅▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁
train/loss,█▆▆▆▅▅▄▆▅▃▇▅▆▆▅▄▃▅▅▅▅▂▅▅▃▅▁▃▃▆▅▄▄▂▃▄▄▅▅▄
total_flos,5.320926935011584e+16
train/epoch,1.99847
train/global_step,1630
train/grad_norm,2.18915
train/learning_rate,0
train/loss,1.9594


In [ ]:
model.config.use_cache = True

In [ ]:
trainer.model.save_pretrained(new_model)
trainer.tokenizer.save_pretrained(new_model)

('/content/drive/MyDrive/Gemma/results/Fine tuning gemma Design/tokenizer_config.json',
 '/content/drive/MyDrive/Gemma/results/Fine tuning gemma Design/special_tokens_map.json',
 '/content/drive/MyDrive/Gemma/results/Fine tuning gemma Design/tokenizer.model',
 '/content/drive/MyDrive/Gemma/results/Fine tuning gemma Design/added_tokens.json',
 '/content/drive/MyDrive/Gemma/results/Fine tuning gemma Design/tokenizer.json')

In [ ]:
MODEL_SAVE_HUB_PATH = f'jsmount/Fine-tuning-gemma-Design'
HUGGINGFACE_AUTH_TOKEN = 'my_token'
trainer.model.push_to_hub(
			MODEL_SAVE_HUB_PATH,
			use_temp_dir=True,
			use_auth_token=HUGGINGFACE_AUTH_TOKEN
)
trainer.tokenizer.push_to_hub(
			MODEL_SAVE_HUB_PATH,
			use_temp_dir=True,
			use_auth_token=HUGGINGFACE_AUTH_TOKEN
)

adapter_model.safetensors:   0%|          | 0.00/41.6M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/jsmount/Fine-tuning-gemma-Design/commit/ff7fdba7c554846f30acd0da1c1cead18c424adc', commit_message='Upload tokenizer', commit_description='', oid='ff7fdba7c554846f30acd0da1c1cead18c424adc', pr_url=None, pr_revision=None, pr_num=None)